In [1]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import pipeline
import numpy as np
import matplotlib.pyplot as plt

c:\ProgramData\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("zero-shot-classification", model=model_name)

c:\ProgramData\miniconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
c:\ProgramData\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zaker\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https:

In [3]:
def classify_sentence(sequence_to_classify, candidate_labels, classifier, threshold, is_multi_label):
  output = classifier(sequence_to_classify, candidate_labels, multi_label = True)
  labels = list(output["labels"])
  scores = list(output["scores"])
  if is_multi_label:
    output_labels = []
    output_scores = []
    for index, score in enumerate(scores):
      if score > threshold:
          output_scores.append(score)
          output_labels.append(labels[index])
    return output_labels, output_scores
  else:
    max_score = 0
    max_index = 0
    for index, score in enumerate(scores):
        if score > max_score:
            max_score = score
            max_index = index
    label = labels[max_index]
    if max_score > threshold:
      return label, max_score
    else:
      return 'None', 0


## Example on how to use the model

In [4]:
labels = ["math", "history", "Arithmetic"]
sentence = "The square root of four is two"

# Define classifier as shown in the second cell. Use threshold = 0.7 
# (Motivated by the results we will show in the presentation)
# Use multi_label == True (last param)
print(classify_sentence(sentence, labels, classifier, 0.7, True))

(['math', 'Arithmetic'], [0.9643085598945618, 0.9238830208778381])
